In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
block_size = 500
torch.manual_seed(101)

filename = 'datasets/text/vietnamese_articles.csv'
df = pd.read_csv(filename)
print(df.head(10))
data = df['Contents']
data.dropna(inplace=True)
data = data.values.tolist()

print(data[:10])

special_token = b'\x03'

for idx in range(len(data)):
    if len(data[idx]) > block_size:
        data[idx] = data[idx][:block_size]
    else:
        data[idx] = data[idx] + ' '* (block_size - len(data[idx]))
    data[idx] = data[idx] 

text = ' '.join(data)
text_set = set(text)

stoi = {ch: i for i, ch in enumerate(sorted(text_set))}
itos = {i: ch for i, ch in enumerate(sorted(text_set))}

vocab_size = len(stoi)
encode = lambda x: torch.tensor([stoi[ch] for ch in x], dtype=torch.long)
decode = lambda x: ''.join([itos[i] for i in x])

n_train = int(len(data) * 0.9)
train_data = [encode(data[i]) for i in range(n_train)]
val_data = [encode(data[i]) for i in range(n_train, len(data))]

print(f'vocab_size = {vocab_size}, n_train = {n_train}')



trang chính  internet society  internet society hay isoc là một tổ chức quốc tế hoạt động phi lợi nhuận phi chính phủ và bao gồm các thành viên có trình độ chuyên ngành tổ chức này chú trọng đến tiêu chuẩn giáo dục và các vấn đề về chính sách với trên một bốn năm tổ chức thành viên và sáu năm không không không thành viên cá nhân isoc bao gồm những con người cụ thể trong cộng đồng internet mọi chi tiết có thể tìm thấy tại website của isoc   internet society nằm ở gần thủ đô washington dc hoa kỳ và geneva thụy sĩ số hội viên của nó bao gồm hơn một bốn năm tổ chức thành viên và hơn sáu năm không không không cá nhân thành viên còn có thể tự lập một chi nhánh của tổ chức tùy theo vị trí hoặc sở thích hiện nay tổ chức có tới chín không chi nhánh trên toàn thế giới   nhiệm vụ và mục đích hoạt động  bảo đảm cổ vũ cho sự phát triển mở rộng và sử dụng internet được thuận lợi nhất cho mọi người trên toàn thế giới   xem thêm   lịch sử internet  liên kết ngoài   isoc việt nam  trang chủ toàn cầu  i

In [3]:
class Head(nn.Module):
    """ one head self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_emb, head_size, bias=False)
        self.query = nn.Linear(n_emb, head_size, bias=False)
        self.value = nn.Linear(n_emb, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        # compute attention scores
        wei = q @ k.transpose(-2, -1) / (C**0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = F.dropout(wei, p=dropout)
        # perform score aggregation
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):

    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
        self.proj = nn.Linear(n_emb, n_emb)

    def forward(self, x):
        x = torch.cat([h(x) for h in self.heads], dim=-1)
        x = self.proj(x)
        x = F.dropout(x, p=dropout)
        return x

class FeedForward(nn.Module):

    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_emb, 4*n_emb),
            nn.ReLU(),
            nn.Linear(4*n_emb, n_emb),
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    """ Transformer Block followed by computation
    """
    def __init__(self, n_emb, n_heads):
        super().__init__()
        self.head_size = n_emb // n_heads
        self.sa = MultiHeadAttention(n_heads, self.head_size)
        self.ff = FeedForward()
        self.ln1 = nn.LayerNorm(n_emb)
        self.ln2 = nn.LayerNorm(n_emb)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        x = F.dropout(x, p=dropout)
        return x
        
class LanguageModel(nn.Module):

    def __init__(self, vocab_size, n_emb):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb)
        self.position_embedding_table = nn.Embedding(block_size, n_emb)
        self.blocks = nn.Sequential(*[Block(n_emb, n_heads) for _ in range(n_layers)])
        self.feed_forward = FeedForward()
        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        token_emb = self.token_embedding_table(idx)
        position_emb = self.position_embedding_table(torch.arange(T, device=device)) 
        x = token_emb + position_emb
        x = self.blocks(x) 
        x = self.feed_forward(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self.forward(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_new = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_new], dim=-1)
        return idx

batch_size = 32
n_emb = 500
n_layers = 6
n_heads = 5
dropout = 0.2
learning_rate = 3e-4

m = LanguageModel(vocab_size=vocab_size, n_emb=n_emb).to(device)
optimizer = torch.optim.Adam(m.parameters(), lr=learning_rate)


In [4]:


def get_random_subtensors(x, block_size, pad_value=0):
    original_len = x.shape[0]
    pad_value = encode(' ').item()
    
    # Check and pad if necessary
    if block_size >= original_len:
        padded_tensor = torch.full((block_size + 1,), pad_value, dtype=x.dtype)
        padded_tensor[:original_len] = x
        x = padded_tensor
    
    # Get a random start index
    start_index = random.randint(0, len(x) - block_size - 1)
    
    # Extract the subtensors
    subtensor1 = x[start_index:start_index + block_size]
    subtensor2 = x[start_index + 1:start_index + block_size + 1]
    
    return subtensor1, subtensor2


def get_batch(data, block_size, batch_size):
    idx = torch.randint(0, len(data)-1, (batch_size,))
    seq = []
    targets = []
    for i in range(batch_size):
        s, t = get_random_subtensors(data[idx[i].item()], block_size)
        seq.append(s)
        targets.append(t)
    x = torch.stack([seq[i] for i in range(batch_size)])
    y = torch.stack([targets[i] for i in range(batch_size)])
    return x, y

def estimate_loss(model, val_data, block_size, batch_size):
    model.eval()
    with torch.no_grad():
        x, y = get_batch(val_data, block_size, batch_size)
        x, y = x.to(device), y.to(device)
        _, loss = model(x, y)
    model.train()
    return loss.item()

x, y = get_batch(train_data, block_size, 32)
print(x.shape)
print(y.shape)

torch.Size([32, 500])
torch.Size([32, 500])


In [5]:
early_stop = 10
last_val_loss = 1e9
n_epochs = 4000

for steps in range(n_epochs):
    xb, yb = get_batch(train_data, block_size, batch_size)
    xb = xb.to(device)
    yb = yb.to(device)
    logits, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if steps % 100 == 0:
        print('Step:', steps, 'Training Loss:', loss.item())
        val_loss = estimate_loss(m, val_data, block_size, batch_size)
        print('Validation loss:', val_loss)
        if val_loss >= last_val_loss:
            early_stop -= 1
            if early_stop == 0:
                print('Early stop!')
                break
        else:
            early_stop = 10
            last_val_loss = val_loss


Step: 0 Training Loss: 6.0964860916137695
Validation loss: 4.826101779937744
Step: 100 Training Loss: 2.4732227325439453
Validation loss: 2.5854413509368896
Step: 200 Training Loss: 2.472409725189209
Validation loss: 2.566506862640381
Step: 300 Training Loss: 2.4316163063049316
Validation loss: 2.55800724029541
Step: 400 Training Loss: 2.4098002910614014
Validation loss: 2.53421688079834
Step: 500 Training Loss: 2.375126838684082
Validation loss: 2.509629726409912
Step: 600 Training Loss: 2.3219563961029053
Validation loss: 2.4438674449920654
Step: 700 Training Loss: 2.270829677581787
Validation loss: 2.309495687484741
Step: 800 Training Loss: 2.159480094909668
Validation loss: 2.2062785625457764
Step: 900 Training Loss: 2.0344676971435547
Validation loss: 2.1869852542877197
Step: 1000 Training Loss: 1.9721791744232178
Validation loss: 2.078052282333374
Step: 1100 Training Loss: 1.9227980375289917
Validation loss: 2.037269353866577
Step: 1200 Training Loss: 1.915468692779541
Validation

In [7]:
print(sum(p.numel() for p in m.parameters() if p.requires_grad))
starting_tokens = 'Việt Nam'
len_starting_tokens = len(starting_tokens)
idx = torch.tensor(encode(starting_tokens)).reshape(1, len_starting_tokens).to(device)
print(decode(m.generate(idx, max_new_tokens=5000)[0].tolist()))

20700918


C:\Users\hvutr\AppData\Local\Temp\ipykernel_7724\2802229414.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idx = torch.tensor(encode(starting_tokens)).reshape(1, len_starting_tokens).to(device)


Việt Nam Giáo dục và Phỏ Giám đối với nồi sau tới đại diện tập triệu lớn phải chính của Trung của Ban Tis Nam Trủ (261 - Môi La Trăng Al) trở giá đốc cần 9 đơn vịch sớm Sài giao số hiêm ngày 22.26,7 điểm khộ giải qua án vong địa và 1960.000 độ C/2020, 81 triệu đồng/kỳ 20220, cùng hỗn mạng việc công tác doanh nghiệp 1015; ngâng là góp tạphổ từ cung phiên tốt tử DN 207 caous với trò, phiên tương; động có 2 người đủ 35-625 - 500-80,05%.000 điểm, và thị tích công tác tiếng, kỹ - xét minh NLĐ. Lạng,                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     